## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [24]:
# 利用numpy 和tensorflow 、pytorch 搭建全连接神经网络。
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################

        # 第一个隐藏层权重与偏置
        self.W1 = tf.Variable(tf.random.normal([28*28, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))

        # 输出层的权重与偏置
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        # 输入层到隐藏层，x形状转换为[batch_size, 28*28]
        x = tf.reshape(x, [-1, 28*28])

        # 第一层    h1 = relu(x*W1 + b1)
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)

        # 输出层    logits = x*W2 + b2
        logits = tf.matmul(h1, self.W2) + self.b2

        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [25]:
@tf.function
def compute_loss(logits, labels):
    '''
    compute loss and accuracy(logits compare with labels?)
    '''
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)    # gradient descent 0.01 before, change to 0.1 could have better performance

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [26]:
train_data, test_data = mnist_dataset() # 获取训练集和测试集

# 训练50个epoch
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
# 测试
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.538133 ; accuracy 0.13455
epoch 1 : loss 2.5114567 ; accuracy 0.1393
epoch 2 : loss 2.4867558 ; accuracy 0.14385
epoch 3 : loss 2.4637377 ; accuracy 0.1488
epoch 4 : loss 2.44217 ; accuracy 0.15416667
epoch 5 : loss 2.4218552 ; accuracy 0.15893333
epoch 6 : loss 2.402635 ; accuracy 0.16456667
epoch 7 : loss 2.3843777 ; accuracy 0.16926667
epoch 8 : loss 2.3669722 ; accuracy 0.17421667
epoch 9 : loss 2.3503194 ; accuracy 0.17958333
epoch 10 : loss 2.3343425 ; accuracy 0.18505
epoch 11 : loss 2.3189719 ; accuracy 0.19078334
epoch 12 : loss 2.30415 ; accuracy 0.19616666
epoch 13 : loss 2.2898266 ; accuracy 0.20215
epoch 14 : loss 2.2759562 ; accuracy 0.20725
epoch 15 : loss 2.2625015 ; accuracy 0.21288334
epoch 16 : loss 2.2494283 ; accuracy 0.21863334
epoch 17 : loss 2.236708 ; accuracy 0.22403333
epoch 18 : loss 2.2243102 ; accuracy 0.22963333
epoch 19 : loss 2.212212 ; accuracy 0.2352
epoch 20 : loss 2.200394 ; accuracy 0.2409
epoch 21 : loss 2.1888375 ; accuracy 0.246